In [2]:
library(dplyr)
library(Seurat)
library(patchwork)
library(ggplot2)

# Setup the Seurat object

In [3]:
inf <- list('GSE134809_69', 'GSE134809_128', 'GSE134809_138', 'GSE134809_158', 'GSE134809_181', 'GSE134809_187', 
            'GSE134809_190', 'GSE134809_193', 'GSE134809_196')
uninf <- list('GSE134809_68', 'GSE134809_129', 'GSE134809_135', 'GSE134809_159', 'GSE134809_180', 'GSE134809_186', 
              'GSE134809_189', 'GSE134809_192', 'GSE134809_195')

In [4]:
for(x in inf) {
    dir <- paste("/data/yebin/singlecell/", x, "/", sep="")
    name <- paste(x, ".data" , sep="")
    assign(name, Read10X(data.dir = dir)) 
    assign(x, CreateSeuratObject(counts = eval(parse(text = name)), project = x, min.cells = 3, min.features = 200)) 
}

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


In [6]:
inf_data <- merge(eval(parse(text = inf[1])), y = c(eval(parse(text = inf[2])),
                                                       eval(parse(text = inf[3])),
                                                       eval(parse(text = inf[4])),
                                                       eval(parse(text = inf[5])),
                                                       eval(parse(text = inf[6])),
                                                       eval(parse(text = inf[7])),
                                                       eval(parse(text = inf[8])),
                                                   eval(parse(text = inf[9]))), 
                      add.cell.ids = c(inf), project = "inflamed", merge.data = TRUE)

In [7]:
for(x in uninf) {
    dir <- paste("/data/yebin/singlecell/", x, "/", sep="")
    name <- paste(x, ".data" , sep="")
    assign(name, Read10X(data.dir = dir)) 
    assign(x, CreateSeuratObject(counts = eval(parse(text = name)), project = x, min.cells = 3, min.features = 200)) 
}

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


In [8]:
uninf_data <- merge(eval(parse(text = uninf[1])), y = c(eval(parse(text = uninf[2])),
                                                       eval(parse(text = uninf[3])),
                                                       eval(parse(text = uninf[4])),
                                                       eval(parse(text = uninf[5])),
                                                       eval(parse(text = uninf[6])),
                                                       eval(parse(text = uninf[7])),
                                                       eval(parse(text = uninf[8])),
                                                       eval(parse(text = uninf[9]))), 
                      add.cell.ids = c(uninf), project = "uninflamed", merge.data = TRUE)

In [9]:
inf_data

An object of class Seurat 
21455 features across 51426 samples within 1 assay 
Active assay: RNA (21455 features, 0 variable features)

In [10]:
uninf_data

An object of class Seurat 
21506 features across 54886 samples within 1 assay 
Active assay: RNA (21506 features, 0 variable features)

# QC

### percentage calculation

In [11]:
inf_data[["percent.mt"]] <- PercentageFeatureSet(inf_data, pattern = "^MT-")
uninf_data[["percent.mt"]] <- PercentageFeatureSet(uninf_data, pattern = "^MT-")
head(uninf_data@meta.data, 5)

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt
,<chr>,<dbl>,<int>,<dbl>
GSE134809_68_AAACATACACCCTC-1,GSE134809_68,659,268,0.7587253
GSE134809_68_AAACATACAGACTC-1,GSE134809_68,717,288,1.6736402
GSE134809_68_AAACATACCAACTG-1,GSE134809_68,2124,537,2.7306968
GSE134809_68_AAACATACCCAACA-1,GSE134809_68,713,356,3.0855540
GSE134809_68_AAACATACCGTCTC-1,GSE134809_68,544,290,3.8602941


In [12]:
epitherial <- c('PLA2G2A', 'CLCA1', 'REG4', 'S100A14', 'ITLN1', 'ELF3', 'PIGR', 'EPCAM', 'REG1B', 'REG1A', 'REG3A', 
                'FABP1', 'RBP2', 'SST', 'FABP2', 'SPINK1', 'FABP6', 'AGR2', 'AGR3', 'CLDN3', 'CLDN4', 'DEFA6', 'DEFA5', 
                'SPINK4', 'ALDOB', 'LCN2', 'KRT8', 'KRT18', 'TSPAN8', 'OLFM4', 'GPX2', 'IFI27', 'PHGR1', 'MT1G',
                'CLDN7', 'KRT19', 'FXYD3', 'LGALS4', 'FCGBP', 'TFF3', 'TFF1')

In [13]:
#All genes must be present to avoid errors
all(epitherial %in% rownames(inf_data))

[1] TRUE

In [14]:
inf_data[["percent.ep"]] <- PercentageFeatureSet(inf_data, features  = epitherial)
uninf_data[["percent.ep"]] <- PercentageFeatureSet(uninf_data, features  = epitherial)
head(uninf_data@meta.data, 5)

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.ep
,<chr>,<dbl>,<int>,<dbl>,<dbl>
GSE134809_68_AAACATACACCCTC-1,GSE134809_68,659,268,0.7587253,0
GSE134809_68_AAACATACAGACTC-1,GSE134809_68,717,288,1.6736402,0
GSE134809_68_AAACATACCAACTG-1,GSE134809_68,2124,537,2.7306968,0
GSE134809_68_AAACATACCCAACA-1,GSE134809_68,713,356,3.0855540,0
GSE134809_68_AAACATACCGTCTC-1,GSE134809_68,544,290,3.8602941,0


In [15]:
hemo <- c("HBB","HBA1","HBA2")

In [16]:
#All genes must be present to avoid errors
all(hemo %in% rownames(inf_data))

[1] TRUE

In [17]:
inf_data[["percent.he"]] <- PercentageFeatureSet(inf_data, features  = hemo)
uninf_data[["percent.he"]] <- PercentageFeatureSet(uninf_data, features  = hemo)
head(uninf_data@meta.data, 5)

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.ep,percent.he
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
GSE134809_68_AAACATACACCCTC-1,GSE134809_68,659,268,0.7587253,0,0
GSE134809_68_AAACATACAGACTC-1,GSE134809_68,717,288,1.6736402,0,0
GSE134809_68_AAACATACCAACTG-1,GSE134809_68,2124,537,2.7306968,0,0
GSE134809_68_AAACATACCCAACA-1,GSE134809_68,713,356,3.0855540,0,0
GSE134809_68_AAACATACCGTCTC-1,GSE134809_68,544,290,3.8602941,0,0


### filter out cell-barcodes

In [18]:
inf_data <- subset(inf_data, subset = nCount_RNA > 800 & percent.ep < 1 & percent.mt < 25 & percent.he < 10 )
uninf_data <- subset(uninf_data, subset = nCount_RNA > 800 & percent.ep < 1 & percent.mt < 25 & percent.he < 10 )

# Data integration

In [22]:
inf_data[['split']] <- 'inf'
uninf_data[['split']] <- 'uninf'

In [23]:
com <- merge(inf_data, y = uninf_data, add.cell.ids = c("inf", "uninf"), project = "infvsuninf")
com

An object of class Seurat 
22450 features across 74414 samples within 1 assay 
Active assay: RNA (22450 features, 0 variable features)

In [26]:
# split the dataset into a list of two seurat objects
com.list <- SplitObject(com, split.by = "split")

# normalize and identify variable features for each dataset independently
com.list <- lapply(X = com.list, FUN = function(x) {
    x <- NormalizeData(x)
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 2000)
})

# select features that are repeatedly variable across datasets for integration
features <- SelectIntegrationFeatures(object.list = com.list)

### perform integration

In [ ]:
immune.anchors <- FindIntegrationAnchors(object.list = com.list, anchor.features = features)

In [ ]:
# this command creates an 'integrated' data assay
immune.combined <- IntegrateData(anchorset = immune.anchors)

In [ ]:
immune.combined

In [ ]:
saveRDS(immune.combined, file = "./immune_intergraion_qc.rds")